In [1]:
using SNNT
using JLD
using Glob

In [16]:
using SNNT.AST
using SNNT.Control

In [39]:
args=Dict(
    "fixed"=>"../../test/parsing/examples/acas/fixed",
    "mapping"=>"../../test/parsing/examples/acas/mapping",
    "formula"=>"../../test/parsing/examples/acas/property-scl2500-compressed"
)

Dict{String, String} with 3 entries:
  "fixed"   => "../../test/parsing/examples/acas/fixed"
  "formula" => "../../test/parsing/examples/acas/property-scl2500-compressed"
  "mapping" => "../../test/parsing/examples/acas/mapping"

In [40]:
fixed_vars_content = open(args["fixed"], "r") do f
    return read(f, String)
end

fixed_parsed = Meta.parse(fixed_vars_content)
fixed_vars = Dict{String,Union{String,Number}}(eval(fixed_parsed))
# Load mapping
mapping_content = open(args["mapping"], "r") do f
    return read(f, String)
end
mapping_parsed = Meta.parse(mapping_content)
mapping = Dict{String,Tuple{SNNT.AST.VariableType,Int64}}(eval(mapping_parsed))
# Load formula
initial_query=load_query(args["formula"],fixed_vars,mapping)
prepared_query=prepare_for_olnnv(initial_query)

Query((-16000.0*x1<=8000.0&16000.0*x1<=8000.0&-40.0*x4<=14.0&40.0*x4<=20.0&-200.0*x2<=100.0&200.0*x2<=100.0&-200.0*x3<=0.0&200.0*x3<=0.0&1.0*x5<=100.0&1.0*x6<=100.0&1.0*x7<=100.0&1.0*x8<=100.0&1.0*x9<=100.0&1.0*x10<=100.0&1.0*x11<=100.0&1.0*x12<=100.0&1.0*x13<=100.0&-1.0*x5<=100.0&-1.0*x6<=100.0&-1.0*x7<=100.0&-1.0*x8<=100.0&-1.0*x9<=100.0&-1.0*x10<=100.0&-1.0*x11<=100.0&-1.0*x12<=100.0&-1.0*x13<=100.0&!(isMax(nnCOCpost,nnCOCpost,nnDNCpost,nnDNDpost,nnDES1500post,nnCLI1500post,nnSDES1500post,nnSCLI1500post,nnSDES2500post,nnSCL2500post))&((-88000.0*x4<=45000.0&0.47*x21.0*x4<-0.51&200.0*x2<0.0)->O((-0.13397549715909093+hNet+(-0.5484375*tNMACNet)+(-0.25568181818181823*vONet)+(-0.53625*(tNMACNet^2.0))+(-0.5*tNMACNet*vONet))<0.0,nothing,nothing))&((((-88000.0*x4<=45000.0&-200.0*x2<=0.0)|(-0.47*x2-1.0*x4<=0.51&200.0*x2<0.0))&((88000.0*x4<=-44000.0&-200.0*x2<=0.0)|(0.47*x21.0*x4<=-0.5&200.0*x2<0.0)))->((16000.0*x1<-100.0&-200.0*x2<=0.0)|(O((100.0+(16000.0*hNet)+(-1864.801864801865*(vONet^2.0)

In [41]:
function get_atoms(formula)
    atoms = Dict{Union{Atom,Predicate,SemiLinearConstraint,LinearConstraint},Bool}()
    if formula isa Atom || formula isa Predicate || formula isa SemiLinearConstraint || formula isa LinearConstraint
        atoms[formula] = true
    elseif istree(formula)
        for a in arguments(formula)
            for f in keys(get_atoms(a))
                atoms[f]=true
            end
        end
    end
    return atoms
end

get_atoms (generic function with 1 method)

In [42]:
function get_tree_depth(formula)
    if istree(formula) && formula isa Formula
        return maximum(get_tree_depth.(arguments(formula)))+1
    else
        return 0
    end
    @warn typeof(formula)
end

get_tree_depth (generic function with 1 method)

In [43]:
print("Number of atoms: ",length(keys(get_atoms(prepared_query.formula))))
print("Tree depth of formula: ",get_tree_depth(prepared_query.formula))

Number of atoms: 112

Tree depth of formula: 9